##Installing Ollama:

In [ ]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (262 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initializ

##Pulling Llama 3.1 70B:

In [ ]:
from IPython.display import clear_output
!ollama pull llama3.1:70b
clear_output()

In [ ]:
!pip install -U lightrag[ollama]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.1/159.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00


##Extracting Data from Txt File:

In [ ]:
def read_file_into_string(file_path):
    try:
        with open(file_path, 'r') as file:
            content = file.read()
            return content
    except FileNotFoundError:
        print(f"File not found at {file_path}")
        return ""

# Example usage
file_path = "/content/textdata5.txt"
content = read_file_into_string(file_path)

##Prompt for creating Question Answer Dataset:

In [ ]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient, GroqAPIClient

import time


qa_template = r"""<SYS>
You are an expert in generating question answer pairs from the text given below:
</SYS>
User: {{input_str}}
Generate 200 theoretical questions and answers for every question and store them in a JSON. Do not repeat questions. Here is a simple format:
{'question': ['what is data analysis?', '' ...],
 'answer': ['Data analysis is ... ', '', ...]}
You:"""

class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({"input_str": str(input)})

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({"input_str": str(input)})

In [ ]:
from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:70b"}
}
# qa = SimpleQA(**model)
# output=qa(content)
# display(Markdown(f"**Answer:** {output.data}"))

##QA Dataset Raw Form:

In [ ]:
qa = SimpleQA(**model)
output=qa(content)
display(Markdown(f"**Answer:** {output.data}"))

**Answer:** Here are 200 theoretical questions and answers related to decision trees, machine learning, and data analysis in the JSON format you requested:

```
{
    "question": [
        "What is a decision tree?",
        "How does a decision tree work?",
        "What is entropy in decision trees?",
        "What is information gain in decision trees?",
        "How do we choose the best split in a decision tree?",
        "What is pruning in decision trees?",
        "What is overfitting in machine learning?",
        "How can we avoid overfitting in decision trees?",
        "What is cross-validation in machine learning?",
        "How does CART work?",
        "What is Gini diversity index in CART?",
        "How do we handle missing data in C4.5 algorithm?",
        "What is the difference between C4.5 and CART algorithms?",
        "Can a decision tree be used for regression tasks?",
        "How can we evaluate the performance of a decision tree?",
        "What is accuracy in machine learning?",
        "What is precision in machine learning?",
        "What is recall in machine learning?",
        "What is F1-score in machine learning?",
        "How do we handle continuous attributes in C4.5 algorithm?",
        "Can a decision tree be used for multi-class classification tasks?",
        "How can we improve the interpretability of a decision tree?",
        "What is feature importance in decision trees?",
        "How can we use decision trees for clustering tasks?",
        "Can a decision tree be used for anomaly detection tasks?",
        "How can we improve the robustness of a decision tree?",
        "What is ensemble learning?",
        "Can we combine multiple decision trees to improve performance?",
        "What is bagging in ensemble learning?",
        "What is boosting in ensemble learning?",
        "How does random forest work?",
        "What is gradient boosting?",
        "How can we tune the hyperparameters of a decision tree?",
        "What is grid search in hyperparameter tuning?",
        "What is random search in hyperparameter tuning?",
        "Can we use decision trees for text classification tasks?",
        "Can we use decision trees for image classification tasks?",
        "How can we improve the efficiency of a decision tree?",
        "What is parallel processing in decision trees?",
        "Can we distribute the computation of a decision tree across multiple machines?",
        "What is distributed computing in decision trees?",
        "How can we visualize the results of a decision tree?",
        "What is visualization in machine learning?",
        "Can we use decision trees for time series forecasting tasks?",
        "Can we use decision trees for recommender systems?",
        "How can we improve the scalability of a decision tree?",
        "What is horizontal scaling in decision trees?",
        "What is vertical scaling in decision trees?",
        "Can we use decision trees for real-time prediction tasks?",
        "How can we improve the reliability of a decision tree?",
        "What is fault tolerance in decision trees?",
        "Can we use decision trees for transfer learning tasks?",
        "Can we use pre-trained decision trees as a feature extractor?",
        "How can we fine-tune a pre-trained decision tree?",
        "What is domain adaptation in decision trees?",
        "Can we use decision trees for few-shot learning tasks?",
        "Can we use decision trees for zero-shot learning tasks?",
        "How can we improve the explainability of a decision tree?",
        "What is feature attribution in decision trees?",
        "Can we use decision trees for model interpretability tasks?",
        "What is model interpretability in machine learning?",
        "Can we use decision trees for uncertainty estimation tasks?",
        "What is uncertainty estimation in machine learning?",
        "How can we improve the robustness of a decision tree to adversarial attacks?",
        "What is adversarial attack in machine learning?",
        "Can we use decision trees for anomaly detection tasks?",
        "Can we use decision trees for outlier detection tasks?",
        "How can we improve the performance of a decision tree on imbalanced datasets?",
        "What is class weighting in decision trees?",
        "What is oversampling in decision trees?",
        "What is undersampling in decision trees?",
        "Can we use decision trees for feature selection tasks?",
        "What is recursive feature elimination in decision trees?",
        "Can we use decision trees for dimensionality reduction tasks?",
        "What is principal component analysis in decision trees?",
        "Can we use decision trees for clustering tasks?",
        "What is k-means clustering in decision trees?",
        "Can we use decision trees for density estimation tasks?",
        "What is kernel density estimation in decision trees?",
        "How can we improve the performance of a decision tree on high-dimensional data?",
        "What is feature hashing in decision trees?",
        "What is random projection in decision trees?",
        "Can we use decision trees for parallel processing tasks?",
        "What is MapReduce in decision trees?",
        "Can we use decision trees for distributed computing tasks?",
        "What is Spark in decision trees?",
        "How can we improve the performance of a decision tree on large-scale data?",
        "What is Hadoop in decision trees?",
        "What is big data in machine learning?",
        "Can we use decision trees for real-time processing tasks?",
        "What is Apache Kafka in decision trees?",
        "Can we use decision trees for streaming data tasks?",
        "What is Apache Flink in decision trees?",
        "How can we improve the performance of a decision tree on IoT devices?",
        "What is edge computing in decision trees?",
        "Can we use decision trees for real-time analytics tasks?",
        "What is Apache Cassandra in decision trees?",
        "Can we use decision trees for NoSQL databases?",
        "What is graph database in decision trees?",
        "How can we improve the performance of a decision tree on complex networks?",
        "What is network analysis in decision trees?",
        "Can we use decision trees for social network analysis tasks?",
        "What is community detection in decision trees?",
        "Can we use decision trees for recommendation systems?",
        "What is collaborative filtering in decision trees?",
        "How can we improve the performance of a decision tree on sequential data?",
        "What is sequence prediction in decision trees?",
        "Can we use decision trees for natural language processing tasks?",
        "What is text classification in decision trees?",
        "Can we use decision trees for image recognition tasks?",
        "What is object detection in decision trees?",
        "How can we improve the performance of a decision tree on audio data?",
        "What is speech recognition in decision trees?"
    ],
    "answer": [
        "A decision tree is a machine learning model that uses a tree-like structure to classify data or make predictions.",
        "A decision tree works by recursively partitioning the data into smaller subsets based on the features of the data.",
        "Entropy is a measure of the uncertainty or randomness in a dataset, and it is used in decision trees to determine the best split.",
        "Information gain is the reduction in entropy that occurs when a decision tree splits the data into two subsets.",
        "The best split in a decision tree is typically chosen based on the information gain, which is calculated using the Gini index or entropy.",
        "Pruning is the process of removing branches from a decision tree to prevent overfitting and improve generalization.",
        "Overfitting occurs when a machine learning model is too complex and fits the training data too closely, resulting in poor performance on new data.",
        "To avoid overfitting in decision trees, we can use techniques such as pruning, regularization, or early stopping.",
        "Cross-validation is a technique used to evaluate the performance of a machine learning model by splitting the data into training and testing sets.",
        "CART (Classification and Regression Trees) is an algorithm for building decision trees that uses a binary split at each node.",
        "The Gini diversity index is a measure of the impurity or heterogeneity of a dataset, and it is used in CART to determine the best split.",
        "In C4.5 algorithm, missing data can be handled using techniques such as imputation, listwise deletion, or pairwise deletion.",
        "C4.5 and CART are both algorithms for building decision trees, but they differ in their handling of continuous attributes and missing values.",
        "Yes, a decision tree can be used for regression tasks by predicting the target variable instead of class labels.",
        "The performance of a decision tree can be evaluated using metrics such as accuracy, precision, recall, F1-score, mean squared error, or R-squared.",
        "Accuracy is the proportion of correct predictions made by a machine learning model.",
        "Precision is the proportion of true positives among all predicted positive instances.",
        "Recall is the proportion of true positives among all actual positive instances.",
        "F1-score is the harmonic mean of precision and recall, providing a balanced measure of both metrics.",
        "In C4.5 algorithm, continuous attributes can be handled using techniques such as binning or discretization.",
        "Yes, a decision tree can be used for multi-class classification tasks by predicting multiple class labels instead of binary class labels.",
        "The interpretability of a decision tree can be improved by visualizing the tree structure, using feature importance scores, or providing explanations for individual predictions.",
        "Feature importance is a measure of the contribution of each feature to the predictions made by a machine learning model.",
        "Yes, a decision tree can be used for clustering tasks by predicting cluster labels instead of class labels.",
        "Anomaly detection involves identifying instances that are significantly different from the majority of the data, and decision trees can be used for this task.",
        "The robustness of a decision tree can be improved using techniques such as pruning, regularization, or ensemble methods.",
        "Ensemble learning is a technique that combines multiple machine learning models to improve performance and robustness.",
        "Yes, we can combine multiple decision trees to improve performance by voting, bagging, or boosting.",
        "The performance of a decision tree on large-scale data can be improved using distributed computing frameworks such as Hadoop or Spark."
    ]
}

In [ ]:
type(output.data)

str

In [ ]:
output.data

'Here are some theoretical questions and answers related to the text you provided:\n\n```\n{\n  "question": [\n    "What is the difference between data science and data analytics?",\n    "What are the types of analytics used in business decision-making?",\n    "What is data literacy and why is it important?",\n    "What is the data ecosystem and lifecycle?",\n    "What is data privacy and what are its key components?",\n    "What is disparate impact and why is it unlawful?"\n  ],\n  "answer": [\n    "Data science focuses on asking questions and finding answers using data, while data analytics is used to extract insights from data to inform business decisions.",\n    "There are four types of analytics: descriptive, diagnostic, predictive, and prescriptive. Each type provides different insights and is used for different purposes.",\n    "Data literacy is the ability to read, understand, and utilize data in different ways. It is important because it enables non-data professionals to make 

##Loading QA Data into proper JSON Format:

In [ ]:
try:
    result = json.loads(output.data)
    # Display the converted JSON
    print("Converted JSON:", json.dumps(result, indent=2))
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    print(f"Output data: {output.data}")

Error decoding JSON: Expecting value: line 1 column 1 (char 0)
Output data: Here are some theoretical questions and answers related to the text you provided:

```
{
  "question": [
    "What is the difference between data science and data analytics?",
    "What are the types of analytics used in business decision-making?",
    "What is data literacy and why is it important?",
    "What is the data ecosystem and lifecycle?",
    "What is data privacy and what are its key components?",
    "What is disparate impact and why is it unlawful?"
  ],
  "answer": [
    "Data science focuses on asking questions and finding answers using data, while data analytics is used to extract insights from data to inform business decisions.",
    "There are four types of analytics: descriptive, diagnostic, predictive, and prescriptive. Each type provides different insights and is used for different purposes.",
    "Data literacy is the ability to read, understand, and utilize data in different ways. It is 

In [ ]:
if output.data.strip():  # Ensures that output is not just whitespace
    try:
        result = json.loads(output.data)
        # Display the converted JSON
        print("Converted JSON:", json.dumps(result, indent=2))
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        print(f"Output data: {output.data}")
else:
    print("Received empty or whitespace-only output from the model.")

Error decoding JSON: Expecting value: line 1 column 1 (char 0)
Output data: Here are some generated theoretical questions and answers related to the text:

[
  {
    "question": ["What is secondary data analysis?", "Why is it important to follow established and systematic procedures when collecting secondary data?"],
    "answer": ["Secondary data analysis is a research methodology that involves analyzing data collected by someone else, often for another purpose.", "It's crucial to follow established procedures to ensure accuracy and reliability of the results."]
  },
  {
    "question": ["What are some common steps in preparing secondary data for statistical analysis?", "How do you collect variables of interest in an established and systematic fashion?"],
    "answer": ["The steps include planning, data collection, data storage, and data cleaning.", "You can collect variables of interest by downloading the necessary data files, collecting only the required variables, and using procedu

In [ ]:
qa_dict

{'question': ['What is the difference between data science and data analytics?',
  'What are the types of analytics used in business decision-making?',
  'What is data literacy and why is it important?',
  'What is the data ecosystem and lifecycle?',
  'What is data privacy and what are its key components?',
  'What is disparate impact and why is it unlawful?'],
 'answer': ['Data science focuses on asking questions and finding answers using data, while data analytics is used to extract insights from data to inform business decisions.',
  'There are four types of analytics: descriptive, diagnostic, predictive, and prescriptive. Each type provides different insights and is used for different purposes.',
  'Data literacy is the ability to read, understand, and utilize data in different ways. It is important because it enables non-data professionals to make informed decisions using data.',
  'The data ecosystem refers to the programming languages, packages, algorithms, cloud-computing serv

In [ ]:
qa_dict['answer']

['Data science focuses on asking questions and finding answers using data, while data analytics is used to extract insights from data to inform business decisions.',
 'There are four types of analytics: descriptive, diagnostic, predictive, and prescriptive. Each type provides different insights and is used for different purposes.',
 'Data literacy is the ability to read, understand, and utilize data in different ways. It is important because it enables non-data professionals to make informed decisions using data.',
 'The data ecosystem refers to the programming languages, packages, algorithms, cloud-computing services, and general infrastructure an organization uses to collect, store, analyze, and leverage data. The data lifecycle describes the path data takes from generation to interpretation.',
 'Data privacy is a subcategory of data protection that encompasses the ethical and legal obligation to protect access to personally identifiable information (PII). Its key components include 

In [ ]:
qa_dict = { "question": [ "What is a decision tree?", "How does a decision tree work?", "What is entropy in decision trees?", "What is information gain in decision trees?", "How do we choose the best split in a decision tree?", "What is pruning in decision trees?", "What is overfitting in machine learning?", "How can we avoid overfitting in decision trees?", "What is cross-validation in machine learning?", "How does CART work?", "What is Gini diversity index in CART?", "How do we handle missing data in C4.5 algorithm?", "What is the difference between C4.5 and CART algorithms?", "Can a decision tree be used for regression tasks?", "How can we evaluate the performance of a decision tree?", "What is accuracy in machine learning?", "What is precision in machine learning?", "What is recall in machine learning?", "What is F1-score in machine learning?", "How do we handle continuous attributes in C4.5 algorithm?", "Can a decision tree be used for multi-class classification tasks?", "How can we improve the interpretability of a decision tree?", "What is feature importance in decision trees?", "How can we use decision trees for clustering tasks?", "Can a decision tree be used for anomaly detection tasks?", "How can we improve the robustness of a decision tree?", "What is ensemble learning?", "Can we combine multiple decision trees to improve performance?", "What is bagging in ensemble learning?", "What is boosting in ensemble learning?", "How does random forest work?", "What is gradient boosting?", "How can we tune the hyperparameters of a decision tree?", "What is grid search in hyperparameter tuning?", "What is random search in hyperparameter tuning?", "Can we use decision trees for text classification tasks?", "Can we use decision trees for image classification tasks?", "How can we improve the efficiency of a decision tree?", "What is parallel processing in decision trees?", "Can we distribute the computation of a decision tree across multiple machines?", "What is distributed computing in decision trees?", "How can we visualize the results of a decision tree?", "What is visualization in machine learning?", "Can we use decision trees for time series forecasting tasks?", "Can we use decision trees for recommender systems?", "How can we improve the scalability of a decision tree?", "What is horizontal scaling in decision trees?", "What is vertical scaling in decision trees?", "Can we use decision trees for real-time prediction tasks?", "How can we improve the reliability of a decision tree?", "What is fault tolerance in decision trees?", "Can we use decision trees for transfer learning tasks?", "Can we use pre-trained decision trees as a feature extractor?", "How can we fine-tune a pre-trained decision tree?", "What is domain adaptation in decision trees?", "Can we use decision trees for few-shot learning tasks?", "Can we use decision trees for zero-shot learning tasks?", "How can we improve the explainability of a decision tree?", "What is feature attribution in decision trees?", "Can we use decision trees for model interpretability tasks?", "What is model interpretability in machine learning?", "Can we use decision trees for uncertainty estimation tasks?", "What is uncertainty estimation in machine learning?", "How can we improve the robustness of a decision tree to adversarial attacks?", "What is adversarial attack in machine learning?", "Can we use decision trees for anomaly detection tasks?", "Can we use decision trees for outlier detection tasks?", "How can we improve the performance of a decision tree on imbalanced datasets?", "What is class weighting in decision trees?", "What is oversampling in decision trees?", "What is undersampling in decision trees?", "Can we use decision trees for feature selection tasks?", "What is recursive feature elimination in decision trees?", "Can we use decision trees for dimensionality reduction tasks?", "What is principal component analysis in decision trees?", "Can we use decision trees for clustering tasks?", "What is k-means clustering in decision trees?", "Can we use decision trees for density estimation tasks?", "What is kernel density estimation in decision trees?", "How can we improve the performance of a decision tree on high-dimensional data?", "What is feature hashing in decision trees?", "What is random projection in decision trees?", "Can we use decision trees for parallel processing tasks?", "What is MapReduce in decision trees?", "Can we use decision trees for distributed computing tasks?", "What is Spark in decision trees?", "How can we improve the performance of a decision tree on large-scale data?", "What is Hadoop in decision trees?", "What is big data in machine learning?", "Can we use decision trees for real-time processing tasks?", "What is Apache Kafka in decision trees?", "Can we use decision trees for streaming data tasks?", "What is Apache Flink in decision trees?", "How can we improve the performance of a decision tree on IoT devices?", "What is edge computing in decision trees?", "Can we use decision trees for real-time analytics tasks?", "What is Apache Cassandra in decision trees?", "Can we use decision trees for NoSQL databases?", "What is graph database in decision trees?", "How can we improve the performance of a decision tree on complex networks?", "What is network analysis in decision trees?", "Can we use decision trees for social network analysis tasks?", "What is community detection in decision trees?", "Can we use decision trees for recommendation systems?", "What is collaborative filtering in decision trees?", "How can we improve the performance of a decision tree on sequential data?", "What is sequence prediction in decision trees?", "Can we use decision trees for natural language processing tasks?", "What is text classification in decision trees?", "Can we use decision trees for image recognition tasks?", "What is object detection in decision trees?", "How can we improve the performance of a decision tree on audio data?", "What is speech recognition in decision trees?" ], "answer": [ "A decision tree is a machine learning model that uses a tree-like structure to classify data or make predictions.", "A decision tree works by recursively partitioning the data into smaller subsets based on the features of the data.", "Entropy is a measure of the uncertainty or randomness in a dataset, and it is used in decision trees to determine the best split.", "Information gain is the reduction in entropy that occurs when a decision tree splits the data into two subsets.", "The best split in a decision tree is typically chosen based on the information gain, which is calculated using the Gini index or entropy.", "Pruning is the process of removing branches from a decision tree to prevent overfitting and improve generalization.", "Overfitting occurs when a machine learning model is too complex and fits the training data too closely, resulting in poor performance on new data.", "To avoid overfitting in decision trees, we can use techniques such as pruning, regularization, or early stopping.", "Cross-validation is a technique used to evaluate the performance of a machine learning model by splitting the data into training and testing sets.", "CART (Classification and Regression Trees) is an algorithm for building decision trees that uses a binary split at each node.", "The Gini diversity index is a measure of the impurity or heterogeneity of a dataset, and it is used in CART to determine the best split.", "In C4.5 algorithm, missing data can be handled using techniques such as imputation, listwise deletion, or pairwise deletion.", "C4.5 and CART are both algorithms for building decision trees, but they differ in their handling of continuous attributes and missing values.", "Yes, a decision tree can be used for regression tasks by predicting the target variable instead of class labels.", "The performance of a decision tree can be evaluated using metrics such as accuracy, precision, recall, F1-score, mean squared error, or R-squared.", "Accuracy is the proportion of correct predictions made by a machine learning model.", "Precision is the proportion of true positives among all predicted positive instances.", "Recall is the proportion of true positives among all actual positive instances.", "F1-score is the harmonic mean of precision and recall, providing a balanced measure of both metrics.", "In C4.5 algorithm, continuous attributes can be handled using techniques such as binning or discretization.", "Yes, a decision tree can be used for multi-class classification tasks by predicting multiple class labels instead of binary class labels.", "The interpretability of a decision tree can be improved by visualizing the tree structure, using feature importance scores, or providing explanations for individual predictions.", "Feature importance is a measure of the contribution of each feature to the predictions made by a machine learning model.", "Yes, a decision tree can be used for clustering tasks by predicting cluster labels instead of class labels.", "Anomaly detection involves identifying instances that are significantly different from the majority of the data, and decision trees can be used for this task.", "The robustness of a decision tree can be improved using techniques such as pruning, regularization, or ensemble methods.", "Ensemble learning is a technique that combines multiple machine learning models to improve performance and robustness.", "Yes, we can combine multiple decision trees to improve performance by voting, bagging, or boosting.", "The performance of a decision tree on large-scale data can be improved using distributed computing frameworks such as Hadoop or Spark." ] }

In [ ]:
# with open("qa_db.json", "w") as outfile:
#     json.dump(qa_dict, outfile)

##Dumping Data into JSON:

In [ ]:
file_path = '/content/qa_db.json'

# Read the existing JSON file
with open(file_path, 'r') as file:
    existing_data = json.load(file)

# If the file contains a list of dictionaries, append the new data
# Otherwise, if it's just a single dictionary, update its keys
if isinstance(existing_data, list):
    existing_data.append(qa_dict)
else:
    # Assuming the existing JSON is a dictionary with 'question' and 'answer' keys
    existing_data['question'].extend(qa_dict['question'])
    existing_data['answer'].extend(qa_dict['answer'])

# Write the updated content back to the JSON file
with open(file_path, 'w') as file:
    json.dump(existing_data, file, indent=4)

print("Data has been added to the JSON file successfully.")

Data has been added to the JSON file successfully.
